灰度值 0-255，黑：0， 白：255

RGB 转灰度值：R，G，B 的比一般为 3：6：1

    1.浮点算法：Gray=R*0.3+G*0.59+B*0.11
    2.整数方法：Gray=(R*30+G*59+B*11)/100
    3.移位方法：Gray =(R*77+G*151+B*28)>>8;
    4.平均值法：Gray=（R+G+B）/3;
    5.仅取绿色：Gray=G；
    
有一个很著名的心理学公式：

    Gray = R*0.299 + G*0.587 + B*0.114
    
而实际应用时，希望避免低速的浮点运算，所以需要整数算法

    Gray = (R*299 + G*587 + B*114 + 500) / 1000

RGB一般是8位精度，现在缩放1000倍，所以上面的运算是32位整型的运算。注意后面那个除法是整数除法，所以需要加上500来实现四舍五入。就是由于该算法需要32位运算，所以该公式的另一个变种很流行：

    Gray = (R*30 + G*59 + B*11 + 50) / 100
    

In [25]:
# 1
from PIL import Image, ImageDraw, ImageFont

ascii_char = list("$@B%8&WM#*oahkbdpqwmZO0QLCJUYXzcvunxrjft/\|()1{}[]?-_+~<>i!lI;:,\"^`'. ")


#将256灰度映射到70个字符上  
def get_char(r, g, b, alpha=256):  #alpha透明度  
    if alpha==0:  
        return ' '  
    length = len(ascii_char)  
    gray = int(0.2126 * r + 0.7152 * g + 0.0722 * b)  #计算灰度  
    unit = (256.0+1) / length  
    return ascii_char[int(gray/unit)]  #不同的灰度对应着不同的字符  
    #通过灰度来区分色块  
  
if __name__=='__main__':
    width = 80
    height = 60
    im = Image.open('ycy.jpg')  
    # im = im.resize((width, height), Image.NEAREST)  # 更改图片的显示比例
    txt = ""  
    for i in range(height):  
        for j in range(width):  
            txt += get_char(*im.getpixel((j, i)))  
        txt += '\n'  
  
    with open("output.txt",'w') as f:  
        f.write(txt)

    im1 = Image.new('L', im.size, 255)
    draw = ImageDraw.Draw(im1)
    draw.text((0, 0), txt, fill=0)
    im1.save('output.jpg')
    im1.close()
    

In [75]:
import os
import bisect

from PIL import Image, ImageDraw, ImageFont


class Img2Txt(object):
    
    def __init__(self):
        self.in_img = 'ycy.jpg'
        # self.out_txt = 'ycy.txt'
        # self.out_img = 'ycy_ascii.jpg'
        self.out_dir = 'dest/'
        self.resolution = 0.3
        self.fnt = ImageFont.truetype('Courier New.ttf', 10)
        self.init_weights()
    
    def init_weights(self):
        chrx, chry = self.fnt.getsize(chr(32))
        normalization = chrx * chry * 255
        
        weights = {}
        # get gray density for characters in range [32, 126]
        for i in range(32, 127):
            chr_image = self.fnt.getmask(chr(i))
            sizex, sizey = chr_image.size
            ctr = sum(chr_image.getpixel((x, y)) for y in range(sizey) for x in range(sizex))
            weights[chr(i)] = ctr / normalization

        weights[chr(32)] = 0.01  # increase it to make blank space ' ' more available
        weights.pop('_', None)  # remove '_' since it is too directional
        weights.pop('-', None)  # remove '-' since it is too directional
        
        self.sorted_weights = sorted(weights.items(), key=operator.itemgetter(1))
        self.scores = [y for (x, y) in self.sorted_weights]
        
        return self.sorted_weights
        
    def get_char(self, val):
        '''
        根据灰度值返回相应的字符串
        '''        
        # find index of val in scores        
        index = bisect.bisect_left(self.scores, val)  

        # check and choose the nearer one between current index and former index
        if index > 0 and self.sorted_weights[index][1] + self.sorted_weights[index - 1][1] > 2 * val:
            index -= 1
            
        return self.sorted_weights[index][0]
    
    def transform(self, img):
        """
        return a string containing characters representing each pixel
        """
        # find index of val in scores
        maximum = self.scores[-1]
        
        img = img.convert("L")  # transform image to black-white
        code = ''
        for h in range(img.size[1]):
            for w in range(img.size[0]):
                gray = img.getpixel((w, h))
                code += self.get_char(maximum * (1 - gray / 255))  # append characters
            code += '\r\n'  # change lines

        return code
    
    def to_txt(self, txt, name='out.txt'):
        with open(os.path.join(self.out_dir, name), 'w') as f:
            f.write(txt)
    
    def to_image(self, txt, src, name='out.png'):
        im = Image.new('L', (int(src.size[0] * sizes[2]), int(src.size[1] * sizes[2])), 255)
        draw = ImageDraw.Draw(im)
        draw.text((0, 0), txt, font=self.fnt, fill=0)
        im.save(os.path.join(self.out_dir, name))
        # im.show()
        im.close()
    
    def start(self):
        src = Image.open(self.in_img)
        sizes = [self.resolution * i for i in (0.665, 0.3122, 4)]
        img = src.resize((int(src.size[0] * sizes[0]), int(src.size[1] * sizes[1])))
        txt = self.transform(img)
        img.close()
        
        # txt
        self.to_txt(txt)
        self.to_image(txt, src)
                  
t = Img2Txt()
t.start()

NameError: name 'sizes' is not defined